In [1]:
import okx.MarketData as MarketData
import csv
import json
from ultils import *
import pandas as pd
import ta
import time

2024-05-31 22:20:00


In [2]:
flag = "0"  # Production trading:0 , demo trading:1
marketDataAPI =  MarketData.MarketAPI(flag=flag)

### input of API is here

In [31]:
# From date and to date
before = "2024-06-01 00:00:00.0" # from
after = "2024-06-15 00:00:00.0" # to 
id = "BTC-USDT" # kind of coin
bar = '2H' # kind of bar 1m, 3m, 5m, 15m, 30m, 1H, 2H, 4H


In [32]:
# Convert bar to timestamp
def convert_bar_to_timestamp(bar):
    bar_to_timestamp = {
        '1m': 1,
        '3m': 3,
        '5m': 5,
        '15m': 15,
        '30m': 30,
        '1H': 60,
        '2H': 120,
        '4H': 240
    }
    
    return bar_to_timestamp.get(bar, "Invalid bar")


timestamp = convert_bar_to_timestamp(bar)

# save the file name
before_name = before
after_name = after

In [33]:
# Convert to miliseconds
before = datetime_to_ms(before)
after = datetime_to_ms(after)
end = after

# Retrieve history candlestick charts from recent years
def get_candlesticks(instId, start_ms, end_ms, bar):
    try:
        result = marketDataAPI.get_history_candlesticks(
            instId=instId,
            after=(start_ms),
            before=(end_ms),
            bar=bar,
        )
        return result
    except Exception as e:
        print(f"Error: {e}")
        return None


### run API and save to .json

In [34]:
# Get history candlesticks
# Schedule to run the script every 10 times per second
result_all = []
while after - before >= 5940000*timestamp:
    result = get_candlesticks(id, after, before, bar)
    result_all.append(result)
    after = after - 5940000*timestamp - 60000*timestamp
    time.sleep(1/20)

result = get_candlesticks(id, after, before, bar)
result_all.append(result)



header:  {'Content-Type': 'application/json', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/market/history-candles?instId=BTC-USDT&after=1718384400000&before=1717174800000&bar=2H
header:  {'Content-Type': 'application/json', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/market/history-candles?instId=BTC-USDT&after=1717664400000&before=1717174800000&bar=2H


In [35]:
# Convert data to DataFrame
rows = []
try:
    for item in result_all:
        for entry in item['data']:
            row = {
                'ts': entry[0],
                'o': entry[1],
                'h': entry[2],
                'l': entry[3],
                'c': entry[4],
                'vol': entry[5],
                'volCcy': entry[6],
                'volCcyQuote': entry[7],
                'confirm': entry[8]
            }
            rows.append(row)

    df = pd.DataFrame(rows)
    # df['ts'] = pd.to_datetime(df['ts'].astype('int64'), unit='ms')
    df[['o', 'h', 'l', 'c', 'vol', 'volCcy', 'volCcyQuote']] = df[['o', 'h', 'l', 'c', 'vol', 'volCcy', 'volCcyQuote']].astype('float64')
except Exception as e:
    print(f"Error: {e}")

In [36]:
# df_indi = ta.add_all_ta_features(df, "o", "h", "l", "c", "vol", fillna=True)
# df_indi.dropna(inplace=True)

In [37]:
filename = f"Dataset_infer/{id}_{bar}_{before_name.replace(' ', '_').replace(':', '-')}_{after_name.replace(' ', '_').replace(':', '-')}.csv"
df.to_csv(filename,index=False)